In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

: 

In [ ]:
df_train = pd.read_csv('./data/train_initial.csv')
del df_train['Unnamed: 0']

In [ ]:
df_train.head()

,id,atm_group,address,address_rus,lat,long,target,city,region
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500,Южно-Сахалинск,Сахалинская область
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710,Южно-Сахалинск,Сахалинская область
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540,Сахалинская область,Сахалинская область
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035,Москва,Москва
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277,Норильск,Красноярский край


In [ ]:
def get_region(row:str) -> str:
    if type(row) == str:
        arr = row.split(',')
        for elem_idx in range(len(arr)):
            if arr[elem_idx].strip() == 'Россия':
                res = arr[elem_idx-1].strip()
                if type(res) == str:
                    return res
                    break
                else:
                    return np.nan
    else:
        return row

In [ ]:
def get_city(row:str) -> str:
    if type(row) == str:
        return row.split(',')[2].strip()
    else:
        return row

In [ ]:
df_train['city'] = df_train['address_rus'].apply(get_city)
df_train['region'] = df_train['address_rus'].apply(get_region)

In [ ]:
df_train = df_train.dropna()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    pd.get_dummies(df_train).drop(['id', 'target'], axis=1),
    pd.get_dummies(df_train)['target'],
    test_size=0.2,
    random_state=13)

In [ ]:
model = LinearRegression()

model.fit(X_train, y_train)
predicts = model.predict(X_test)

In [ ]:
mean_absolute_error(y_test, predicts)

80766.8346904284